In [75]:
from pyspark.sql import SparkSession
from datetime import datetime as dt, timedelta
import pandas as pd
import re

spark = SparkSession.builder.master('local[2]').appName('airflow log test').getOrCreate()
data = spark.read.text("MySQL_to_CSV.text") # change pat
    
# value 열에서 데이터 추출
sample_logs = [item['value'] for item in data.collect()]

# 정규식 사용해 timestamp 추출 
ts_pattern = re.compile(r'\[(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3})]')

# timestamp가 존재하는 열 추출 
time_ext_log = [item for item in sample_logs if ts_pattern.match(item)]


# timestamp parsing 
temp_datetime = [ts_pattern.match(item).group(1) for item in time_ext_log if ts_pattern.match(item)]
dates = pd.to_datetime(temp_datetime, format='%Y-%m-%d %H:%M:%S')

datetime = dates.to_list()

# Status Prsing List 
log_level = list()

# Message Parsing List 
message = list()

for row in time_ext_log:
    
    if "INFO" in row :
        log_level.append("INFO") # Status Parsing 
        message.append(row.split("INFO - ")[1]) # Message Parsing 
        
    elif "WARNING" in row :
        log_level.append("WARNING") # Status Parsing 
        message.append(row.split("WARNING - ")[1])  # Message Parsing 
        
    elif "ERROR" in row :
        log_level.append("ERROR") # Status Parsing 
        message.append(row.split("ERROR - ")[1])  # Message Parsing 
        
        
df = pd.DataFrame({'datetime' : datetime, 'log-level':log_level, 'message':message})  # 데이터 프레임 생성 
js = df.to_json(orient = 'records')     # json 파일 변경 

ValueError: time data 2022-01-19 11:46:20,775 doesn't match format specified

In [57]:
df.head(5)

,datetime,log-level,message
0,"2022-01-19 11:46:20,775",INFO,Started process (PID=946) to work on /opt/airf...
1,"2022-01-19 11:46:20,780",INFO,Processing file /opt/airflow/dags/MySQL_to_CSV...
2,"2022-01-19 11:46:20,783",INFO,"[2022-01-19 11:46:20,782] {dagbag.py:500}"
3,"2022-01-19 11:46:22,634",INFO,DAG(s) dict_keys(['second_assingnment']) retri...
4,"2022-01-19 11:46:27,461",INFO,"[2022-01-19 11:46:27,461] {providers_manager.p..."


In [58]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch()

In [59]:
print(dt.now().strftime('%Y-%m-%d'))

2022-02-05


In [60]:
index_name = 'airflow-web-log_' + dt.now().strftime('%Y-%m-%d')
index_name

'airflow-web-log_2022-02-05'

In [52]:
def bulk_es(df):
    for index, row in df.iterrows():
        yield {
            '_index': index_name,
            '_type': '_doc',
            '_source': filterKeys(row)
        }
    raise StopIterationb
    
JsonKey = ['datetime', 'log-level','message']
def filterKeys(rows):
    return {key: rows[key] for key in JsonKey}

In [63]:
try:
    res = helpers.bulk(es, bulk_es(df))
    print("Working")
except Exception as e:
    print(e)

/opt/homebrew/anaconda3/envs/pipeline/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


name 'StopIterationb' is not defined


In [76]:
spark.stop()